# Food Delivery Operations Analytics

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/raw/zomato_dataset.csv")
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41


In [2]:
# Data overview
print("Show dataset shape")
print("Dataset shape:", df.shape)

print("\nShow collumns")
print("Columns:", df.columns.tolist())

print("\nSummary statistics")
print(df.describe())

print("\nCheck missing values")
print(df.isnull().sum())

print("\nData types")
print(df.dtypes)

print("\nDistribution of categorical columns")
print(df['Type_of_order'].value_counts())
print(df['City'].value_counts())
print(df['Festival'].value_counts())

Show dataset shape
Dataset shape: (45584, 20)

Show collumns
Columns: ['ID', 'Delivery_person_ID', 'Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude', 'Order_Date', 'Time_Orderd', 'Time_Order_picked', 'Weather_conditions', 'Road_traffic_density', 'Vehicle_condition', 'Type_of_order', 'Type_of_vehicle', 'multiple_deliveries', 'Festival', 'City', 'Time_taken (min)']

Summary statistics
       Delivery_person_Age  Delivery_person_Ratings  Restaurant_latitude  \
count         43730.000000             43676.000000         45584.000000   
mean             29.566911                 4.633774            17.017948   
std               5.815064                 0.334744             8.185674   
min              15.000000                 1.000000           -30.905562   
25%              25.000000                 4.500000            12.933284   
50%              30.000000                 4.700000

In [13]:
# Data Cleaning
import numpy as np

df['Delivery_person_Age'] = df['Delivery_person_Age'].fillna(df['Delivery_person_Age'].median())
df['Delivery_person_Ratings'] = df['Delivery_person_Ratings'].fillna(df['Delivery_person_Ratings'].median())

df['Weather_conditions'] = df['Weather_conditions'].fillna('Unknown')
df['Road_traffic_density'] = df['Road_traffic_density'].fillna('Unknown')
df['Festival'] = df['Festival'].fillna('Unknown')
df['City'] = df['City'].fillna('Unknown')

mode_val = df['multiple_deliveries'].mode()[0]
df['multiple_deliveries'] = df['multiple_deliveries'].fillna(mode_val)

# Data Cleaning for Time_Orderd 
def fix_time_format(x):
    if pd.isna(x):
        return np.nan
    try:
        parts = str(x).split(':')
        if len(parts) >= 2:
            h, m = int(parts[0]), int(parts[1])
            h = h % 24 
            return f"{h:02d}:{m:02d}"
        return np.nan
    except:
        return np.nan

df['Time_Orderd'] = df['Time_Orderd'].apply(fix_time_format)
df['Time_Order_picked'] = df['Time_Order_picked'].apply(fix_time_format)

# คำนวณ Time_taken_actual สำหรับ row ที่มี Time_Orderd
mask_known = df['Time_Orderd'].notna()
df.loc[mask_known, 'Time_taken_actual'] = (
    pd.to_datetime(df.loc[mask_known, 'Time_Order_picked'], format='%H:%M')
    - pd.to_datetime(df.loc[mask_known, 'Time_Orderd'], format='%H:%M')
).dt.total_seconds() / 60

group_median = df[mask_known].groupby(['Type_of_order','Type_of_vehicle'])['Time_taken_actual'].median()

mask_missing = df['Time_Orderd'].isna()
for idx in df[mask_missing].index:
    order_type = df.at[idx, 'Type_of_order']
    vehicle_type = df.at[idx, 'Type_of_vehicle']
    if (order_type, vehicle_type) in group_median.index:
        median_time = group_median.loc[(order_type, vehicle_type)]
        picked_time = pd.to_datetime(df.at[idx, 'Time_Order_picked'], format='%H:%M', errors='coerce')
        
        if pd.notna(picked_time):
            new_time = picked_time - pd.Timedelta(minutes=median_time)
            if pd.notna(new_time): 
                df.at[idx, 'Time_Orderd'] = new_time.strftime('%H:%M')
                
# numeric summary
print(df['Delivery_person_Age'].describe())
print(df['Delivery_person_Ratings'].describe())

# categorical summary
print(df['multiple_deliveries'].value_counts())
print(df['Weather_conditions'].value_counts())
print(df['Road_traffic_density'].value_counts())
print(df['Festival'].value_counts())
print(df['City'].value_counts())
print(df['Time_Orderd'].head())

count    45584.000000
mean        29.584525
std          5.696221
min         15.000000
25%         25.000000
50%         30.000000
75%         34.000000
max         50.000000
Name: Delivery_person_Age, dtype: float64
count    45584.000000
mean         4.636546
std          0.327931
min          1.000000
25%          4.600000
50%          4.700000
75%          4.800000
max          6.000000
Name: Delivery_person_Ratings, dtype: float64
multiple_deliveries
1.0    29144
0.0    14094
2.0     1985
3.0      361
Name: count, dtype: int64
Weather_conditions
Fog           7653
Stormy        7584
Cloudy        7533
Sandstorms    7494
Windy         7422
Sunny         7282
Unknown        616
Name: count, dtype: int64
Road_traffic_density
Low        15476
Jam        14139
Medium     10945
High        4423
Unknown      601
Name: count, dtype: int64
Festival
No         44460
Yes          896
Unknown      228
Name: count, dtype: int64
City
Metropolitian    34087
Urban            10133
Unknown        

In [1]:
import pandas as pd

df = pd.read_csv("../data/cleaned_zomato.csv")
df = df.rename(columns={'Time_Orderd': 'Time_Ordered'})
df.to_csv("../data/raw/cleaned_zomato.csv", index=False)